# MobileNetV2

In [1]:
import os
import shutil
import random
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
# Define parameters
batch_size = 32
epochs = 29
image_size = (224, 224)  # Image size expected by MobileNetV2
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

In [3]:
# Define parameters
batch_size = 32
epochs = 29
image_size = (224, 224)  # Image size expected by MobileNetV2
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

Found 5505 images belonging to 11 classes.
Found 1372 images belonging to 11 classes.


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'C:\dataset',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # use 'categorical' for multiple classes
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'C:\dataset',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # use 'categorical' for multiple classes
    subset='validation'
)


In [4]:
# Load MobileNetV2 pre-trained on ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

In [5]:
# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x) 
predictions = Dense(11, activation='softmax')(x)  # Adjust output neurons based on your problem (11 classes)

In [6]:
# Combine base model and custom head
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
# Freeze layers in the base model
for layer in base_model.layers:
    layer.trainable = False

In [8]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # use 'categorical_crossentropy' for multiple classes
              metrics=['accuracy'])

In [9]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/29


172/172 [==============================] - 152s 850ms/step - loss: 1.7708 - accuracy: 0.3808 - val_loss: 0.8726 - val_accuracy: 0.6830
Epoch 2/29
172/172 [==============================] - 99s 578ms/step - loss: 0.9356 - accuracy: 0.6629 - val_loss: 0.5947 - val_accuracy: 0.7887
Epoch 3/29
172/172 [==============================] - 133s 772ms/step - loss: 0.7507 - accuracy: 0.7416 - val_loss: 0.5762 - val_accuracy: 0.7961
Epoch 4/29
172/172 [==============================] - 144s 835ms/step - loss: 0.6480 - accuracy: 0.7738 - val_loss: 0.5295 - val_accuracy: 0.8214
Epoch 5/29
172/172 [==============================] - 254s 1s/step - loss: 0.5740 - accuracy: 0.8034 - val_loss: 0.4970 - val_accuracy: 0.8333
Epoch 6/29
172/172 [==============================] - 109s 636ms/step - loss: 0.5244 - accuracy: 0.8204 - val_loss: 0.5114 - val_accuracy: 0.8170
Epoch 7/29
172/172 [==============================] - 98s 568ms/step - loss: 0.4908 - accuracy: 0.8288 - val_loss: 0.5129 - va

In [10]:
# Evaluate the model on validation set
loss, accuracy = model.evaluate(validation_generator)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)


43/43 [==============================] - 33s 747ms/step - loss: 0.5872 - accuracy: 0.8542
Validation Loss: 0.5871989727020264
Validation Accuracy: 0.8542274236679077


In [12]:
# Save the model
model.save('MobileNetV2_model.h5')

#save history
import pickle
with open('MobileNetV2_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)